In [1]:
import pandas as pd
import numpy as np

## Loops

In [2]:
import random
def average_py(n):
    s = 0
    for i in range(n):
        s += random.random()
    return s / n

In [3]:
n = 1000000

In [4]:
%time average_py(n)

CPU times: user 99.4 ms, sys: 2.35 ms, total: 102 ms
Wall time: 101 ms


0.49951423343146406

In [2]:
# placeholder for optimization alg

In [ ]:
# check for expected sum here